## External Sources: Weather

This notebook retrieves the weather data for the 5 cities.

In [3]:
city = 'Paris'

In [4]:
import collections
import datetime as dt
import dateutil.parser as dp
import logging

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

ImportError: No module named 'tqdm'

In [ ]:
% cd ../..
import src
% cd notebooks/ext

In [ ]:
# Load the order dates to determine the time
# span for which the weather data is needed.
df = pd.read_csv(
    '../../data/interim/{}_1_orders.csv.bz2'
        .format(city.lower()),
    usecols=['placed_at'],
    parse_dates=['placed_at'],
)

# Retrieve weather data starting 6 weeks
# before the order data to be able to
# calculate trailing averages.
start_day = (df['placed_at'].min().date()
             - dt.timedelta(weeks=6))
end_day = (df['placed_at'].max().date()
           + dt.timedelta(days=1))
del df

print('Weather data is extracted from {} to {}.'
          .format(start_day, end_day))

# Define data structures to store the results.

sunsets = []
Sunset = collections.namedtuple('Sunset', (
             'date', 'civil', 'nautical'))

moons = []
Moon = collections.namedtuple('Moon', (
           'date', 'description', 'illumination'))

weather = []
Weather = collections.namedtuple('Weather', (
             'datetime', 'temperature', 'humidity',
             'wind', 'visibility', 'conditions'))

# Iterate over every day for which the weather is to be retrieved.
for day in tqdm_notebook(pd.date_range(start_day, end_day, freq='D'),
                         total=((end_day - start_day).days + 1)):
    date = dt.date(day.year, day.month, day.day)

    # Retrieve the raw data.
    url = ('https://www.wunderground.com/history/'
           'airport/{}/{}/{}/{}/DailyHistory.html'
               .format(src.constants.cities[city]['airport_code'],
                       day.year, day.month, day.day))
    html = pd.read_html(url)

    # Parse the table that contains the times
    # of the civil and nautical sunsets.

    sunset = html[2]

    assert sunset.shape == (4, 3)
    assert sunset.iloc[1, 0] == 'Civil Twilight'
    assert sunset.iloc[2, 0] == 'Nautical Twilight'
    assert ('CET' in sunset.iloc[1, 2]
            or 'CEST' in sunset.iloc[1, 2])
    assert ('CET' in sunset.iloc[2, 2]
            or 'CEST' in sunset.iloc[2, 2])
    
    civil = (dp.parse(sunset.iloc[1, 2].split(' ')[0])
             + dt.timedelta(hours=12)).time()
    nautical = (dp.parse(sunset.iloc[2, 2].split(' ')[0])
                + dt.timedelta(hours=12)).time()

    sunsets.append(Sunset(date, civil, nautical))

    # Parse the table that contains the info
    # about the moon (descriptive and illumination).

    moon = html[3]

    assert moon.shape == (1, 1)

    parts = moon.iloc[0, 0].split(', ')
    moon_desc = parts[0]

    parts = parts[1].split('%')
    moon_illumination = int(parts[0])
    
    moons.append(Moon(date, moon_desc, moon_illumination))

    # Parse the table that contains the
    # hourly weather data.

    try:
        hourly_weather = html[4]
    except IndexError:
        logging.warning(' {}: 24 hours are not covered.'
                            .format(date))
        continue

    hours = set()

    # Iterate over the hours of a day.
    # Note: Depending on the city the data comes
    # either in 60 minutes or even 30 minutes
    # intervals. Moreover, some cities have
    # two sources so that for every hour there
    # can be more than one entry.
    for _, row in hourly_weather.iterrows():

        # Parse the local time out of the table.
    
        try:
            time, half = row['Time (CET)'].split(' ')
        except KeyError:
            time, half = row['Time (CEST)'].split(' ')

        hour, minute = time.split(':')
        hour, minute = int(hour), int(minute)

        if half == 'AM' and hour == 12:
            hour = 0
        elif half == 'PM' and hour < 12:
            hour += 12
        
        hours.add(hour)

        datetime = dt.datetime(day.year, day.month, day.day,
                               hour, minute)

        # Parse the temperature out of the HTML.
        # Note: depending on the location of the
        # computer running this notebook, the data
        # is retrieved in either metric or US units.
        # Non-metric units are converted into metric ones.
        assert 'F' in row['Temp.']
        temperature = round((float(row['Temp.']
                                       .strip(' °F'))
                             - 32)
                            * 5 / 9, 1)

        # Parse the humidity out of the HTML.
        assert '%' in row['Humidity']
        humidity = int(row['Humidity']
                           .strip('%'))

        # Parse the wind speed out of the HTML.
        # Note: conversion into metric units
        # must be done here too.
        if row['Wind Speed'] == 'Calm':
            wind = 1
        elif row['Wind Speed'] == '-':
            wind = np.NaN
        else:
            assert 'mph' in row['Wind Speed']
            wind = round(float(row['Wind Speed']
                                   .strip(' mph'))
                         * 1.609344, 1)
            assert wind > 1

        # Parse the visibility out of the HTML.
        # Note: conversion must be done here too.
        if row['Visibility'] != '-':
            assert 'mi' in row['Visibility']
            visibility = round(float(row['Visibility']
                                         .strip(' mi'))
                               * 1.609344, 1)
        else:
            visibility = np.NaN

        # Parse the generic description of the
        # weather out of the HTML.
        if type(row['Conditions']) == type(np.NaN):
            conditions = 'unknown'
        else:
            conditions = row['Conditions'].lower()

        # Store the result object.
        weather.append(Weather(
            datetime, temperature, humidity,
            wind, visibility, conditions
        ))

    if len(hours) < 24:
        logging.warning(' {}: {:d} hours are not covered.'
                            .format(date, 24 - len(hours)))


# Create DataFrames out of the
# individual data objects.
sunsets = pd.DataFrame(sunsets)
moons = pd.DataFrame(moons)
weather = pd.DataFrame(weather)

# For visibility and generic weather descriptions often times
# only one of two sources (in cities with two sources) provides
# data. The subsequent code fills up missing values for rows
# representing the same time window.

weather['previous_datetime'] = weather['datetime'].shift(1)
weather['previous_visibility'] = weather['visibility'].shift(1)
weather['previous_conditions'] = weather['conditions'].shift(1)

mask = (weather['previous_datetime']
        == weather['datetime'])

mask2 = mask & weather['visibility'].isnull()
weather.loc[mask2,
            'visibility'] = weather.loc[mask2,
                                        'previous_visibility']

mask2 = mask & (weather['conditions'] == 'unknown')
weather.loc[mask2,
            'conditions'] = weather.loc[mask2,
                                        'previous_conditions']

# Finally, redundant rows are discarded.
# The source "METAR" offers 30 minutes time windows while
# "AAXX" only offers 60 minutes windows. The discarding
# is done such that "METAR" entries are kept with priority.

weather['subsequent_datetime'] = weather['datetime'].shift(-1)

mask = (weather['subsequent_datetime']
        == weather['datetime'])

weather = weather[~mask]

# For unknown reasons, there are still duplicates
# left after the previous step. This takes care of it.
weather.drop_duplicates('datetime', inplace=True)

# Delete temporary columns.
del weather['previous_datetime']
del weather['previous_visibility']
del weather['previous_conditions']
del weather['subsequent_datetime']

# Set a meaningful index.
sunsets.set_index('date', inplace=True)
moons.set_index('date', inplace=True)
weather.set_index('datetime', inplace=True)

# Reindex the DataFrames to ensure every
# time slot is actually reflecting even if
# no weather data could be retrieved.

index = pd.date_range(start_day, end_day, freq='D')

sunsets = sunsets.reindex(index)
sunsets.index.name = 'date'

moons = moons.reindex(index)
moons.index.name = 'date'

index = pd.date_range(start_day, end_day, freq='30T')

weather = weather.reindex(index)
weather.index.name = 'datetime'

In [ ]:
sunsets.info()

In [ ]:
sunsets.head(10)

In [ ]:
sunsets.to_csv(
    '../../data/external/{}_0_sunsets.csv.bz2'
        .format(city.lower()),
    compression='bz2')

In [ ]:
moons.info()

In [ ]:
moons.head(10)

In [ ]:
moons.to_csv(
    '../../data/external/{}_0_moons.csv.bz2'
        .format(city.lower()),
    compression='bz2')

In [ ]:
weather.info()

In [ ]:
weather.head(10)

In [ ]:
weather.to_csv(
    '../../data/external/{}_0_weather.csv.bz2'
        .format(city.lower()),
    compression='bz2')